# #011 - Portfolio Optimization

## libs

In [1]:
#import Libraries
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import random

## Functions

In [2]:
def rand_weights(n):
    ''' Produces n random weights that sum to 1 '''
    k = np.random.rand(n)
    return k / sum(k)

In [3]:
def run_portfolio(df, weights, initial_investment, risk_free):
    ''' Performs an asset allocation and calc Portfolio Metrics'''
    # The function returns: 
    # (0) Portfolio Data Frame 
    # (1) Expected portfolio return 
    # (2) Expected volatility 
    # (3) Sharpe ratio 
    # (4) Return on investment 
    # (5) Final portfolio value in dollars
    
    # Perform asset allocation using the random weights (sent as arguments to the function)
    portfolio_df =  df.copy()
    # Scale stock prices using the "price_scaling"
    scaled_df = df.copy()
    for i in df.columns[0:]:
        scaled_df[i] = df[i]/df[i][0]
    #enumerate method links Stocks tickers in columns along with a counter position weight (i), like an index
    for i, stock in enumerate(scaled_df):
        portfolio_df[stock] = weights[i] * scaled_df[stock]  * initial_investment
    
    # Sum up all values and place the result in a new column titled "portfolio value [$]" 
    portfolio_df['Total Value [$]'] = portfolio_df.sum(axis = 1, numeric_only = True)
    # Calculate the portfolio percentage daily return and replace NaNs with zeros
    portfolio_df['Daily Return [%]'] = portfolio_df['Total Value [$]'].pct_change(1) * 100 
    portfolio_df.replace(np.nan, 0, inplace = True)
    # Calculate the return on the investment = last final value of the portfolio compared to its initial value
    return_on_investment = ((portfolio_df['Total Value [$]'][-1:] - portfolio_df['Total Value [$]'][0]) 
                            / portfolio_df['Total Value [$]'][0]) * 100
    
    # Daily change of every stock in the portfolio 
    portfolio_daily_return_df = portfolio_df.drop(columns = ['Total Value [$]', 'Daily Return [%]'])
    portfolio_daily_return_df = portfolio_daily_return_df.pct_change(1)
    portfolio_daily_return_df.replace(np.nan, 0, inplace = True)  
  
    # Portfolio Expected Return formula
    expected_portfolio_return = np.sum(weights * portfolio_daily_return_df.mean() ) * 252
  
    # Portfolio standard deviation, volatility (risk)
    covariance = portfolio_daily_return_df.cov() * 252 
    expected_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))
    
    # Calculate Sharpe ratio
    sharpe_ratio = (expected_portfolio_return - risk_free)/expected_volatility 
    
    return expected_portfolio_return, expected_volatility, sharpe_ratio, portfolio_df['Total Value [$]'][-1:].values[0], return_on_investment.values[0]

## 11.1 Read and Organize Data and Run Simulations

In [4]:
# file_name =          input('Input the CSV file name: ')
# initial_investment = int(input('Input the initial investment: '))
# risk_free =          float(input('Input the risk free annual rate: ')) # https://ycharts.com/ind icators/10_year_treasury_rate 
# sim_runs =           int(input('Input how many runs to perform on Monte Carlo Simulation: '))

file_name =          'MAG7'
initial_investment = 1000000
risk_free =          0.039
sim_runs =           1000

In [5]:
# read CSV file
df = pd.read_csv(file_name)
# replace the colum Date to Index
df.set_index(['Date'], inplace = True)

In [6]:
# Set the number of simulation runs
n = n_assets = len(df.columns)

# Array to store all weights
weights_runs = np.zeros((sim_runs, n))
# Array to store all Sharpe ratios
sharpe_ratio_runs = np.zeros(sim_runs)
# Array to store all expected returns
expected_portfolio_returns_runs = np.zeros(sim_runs)
# Array to store all volatility values
volatility_runs = np.zeros(sim_runs)
# Array to store all returns on investment
return_on_investment_runs = np.zeros(sim_runs)
# Array to store all final portfolio values
final_value_runs = np.zeros(sim_runs)

# RUN MONTE CARLO SIMS
for i in range(sim_runs):
    # Generate random weights 
    weights = rand_weights(n)
    # Store the weights
    weights_runs[i,:] = weights
    
    # Call 'run_portfolio' function and store Sharpe ratio, return and volatility 
    expected_portfolio_returns_runs[i], volatility_runs[i], sharpe_ratio_runs[i], final_value_runs[i], return_on_investment_runs[i] = run_portfolio(df, weights, initial_investment, risk_free)
    print("Simulation Run = {}".format(i))   
    print("Weights = {}, Final Value = ${:.2f}, Sharpe Ratio = {:.2f}".format(weights_runs[i].round(3), final_value_runs[i], sharpe_ratio_runs[i]))   
    print('\n')

Simulation Run = 0
Weights = [0.063 0.133 0.191 0.177 0.203 0.125 0.108], Final Value = $4895060.84, Sharpe Ratio = 0.98


Simulation Run = 1
Weights = [0.057 0.09  0.108 0.206 0.231 0.298 0.01 ], Final Value = $6065196.17, Sharpe Ratio = 1.02


Simulation Run = 2
Weights = [0.075 0.241 0.171 0.028 0.021 0.256 0.207], Final Value = $6773490.91, Sharpe Ratio = 1.09


Simulation Run = 3
Weights = [0.037 0.246 0.009 0.243 0.182 0.2   0.082], Final Value = $5323564.44, Sharpe Ratio = 0.96


Simulation Run = 4
Weights = [0.175 0.151 0.154 0.137 0.184 0.102 0.098], Final Value = $4817427.97, Sharpe Ratio = 1.00


Simulation Run = 5
Weights = [0.156 0.117 0.061 0.18  0.166 0.141 0.179], Final Value = $5762343.27, Sharpe Ratio = 1.07


Simulation Run = 6
Weights = [0.193 0.078 0.153 0.151 0.151 0.121 0.152], Final Value = $5461456.85, Sharpe Ratio = 1.07


Simulation Run = 7
Weights = [0.198 0.01  0.251 0.142 0.029 0.172 0.198], Final Value = $6246601.76, Sharpe Ratio = 1.11


Simulation Run =

## 11.2 Portfolio Optimization